In [6]:
import transformers
import pandas as pd
import json
import os

## resources for SBERT
https://github.com/UKPLab/sentence-transformers/blob/master/examples/training/sts/training_stsbenchmark.py 
https://www.sbert.net/examples/training/sts/README.html#loss-function 
https://github.com/cerlymarco/MEDIUM_NoteBook/blob/master/Siamese_Dual_BERT/Siamese_Dual_BERT.ipynb
    

## start by getting the data in the right format for training 

In [7]:
outerFolder = "/shared/3/projects/benlitterer/agendaSetting/trainingHtml"

dfList = []
for folderNum in os.listdir(outerFolder): 
    innerFolder = outerFolder + "/" + folderNum 
    for innerFileName in os.listdir(innerFolder): 
        innerPathName = outerFolder + "/" + folderNum + "/" + innerFileName
        if "json" in innerPathName: 
            with open(innerPathName, "r") as f: 
                data = json.loads(f.read())
            df = pd.json_normalize(data)
            df["id"] = innerFileName.split(".")[0]
            dfList.append(df)
        

In [8]:
fullDf = pd.concat(dfList).reset_index(drop=True)

In [22]:
leanDf = fullDf[["id", "title", "url", "source_url", "text"]]

In [23]:
groundTruths = pd.read_csv("/shared/3/projects/benlitterer/agendaSetting/trainingDataV2.csv")
print(groundTruths.shape)
groundTruths.head(3)

(4964, 14)


,url1_lang,url2_lang,pair_id,link1,link2,ia_link1,ia_link2,Geography,Entities,Time,Narrative,Overall,Style,Tone
0,en,en,1484084337_1484110209,https://www.washingtonpost.com/local/virginia-...,https://www.washingtonpost.com/world/the_ameri...,https://web.archive.org/web/www.washingtonpost...,https://web.archive.org/web/www.washingtonpost...,4.0,4.0,1.0,4.000000,4.000000,1.666667,2.000000
1,en,en,1484396422_1483924666,https://www.stlucianewsonline.com/guyana-three...,https://www.thestar.com/news/world/europe/2020...,https://web.archive.org/web/www.stlucianewsonl...,https://web.archive.org/web/www.thestar.com/ne...,4.0,4.0,1.0,4.000000,3.666667,1.666667,1.333333
2,en,en,1484698254_1483758694,https://www.teaparty.org/trump-brings-in-2020-...,https://www.timesofisrael.com/trump-says-he-do...,https://web.archive.org/web/www.teaparty.org/t...,https://web.archive.org/web/www.timesofisrael....,1.0,2.0,1.0,2.333333,2.333333,1.000000,1.333333


In [24]:
#groundTruths.loc[(groundTruths["url1_lang"] == "en") & (groundTruths["url2_lang"] == "en")].shape

In [25]:
groundTruths[["id1", "id2"]] = groundTruths["pair_id"].str.split("_", expand=True)

In [26]:
leanDf = leanDf[["id", "text", "title"]]

In [28]:
#merge text for the first id, id1
leanDf1 = leanDf.rename(columns = {"id":"id1","text":"text1", "title":"title1"})
firstMerge = pd.merge(groundTruths, leanDf1, how="left", on="id1")
firstMerge.head(3)

,url1_lang,url2_lang,pair_id,link1,link2,ia_link1,ia_link2,Geography,Entities,Time,Narrative,Overall,Style,Tone,id1,id2,text1,title1
0,en,en,1484084337_1484110209,https://www.washingtonpost.com/local/virginia-...,https://www.washingtonpost.com/world/the_ameri...,https://web.archive.org/web/www.washingtonpost...,https://web.archive.org/web/www.washingtonpost...,4.0,4.0,1.0,4.000000,4.000000,1.666667,2.000000,1484084337,1484110209,"MARTINSBURG, W.Va. — A suspected drunken drive...",Virginia man arrested in fatal DUI crash in We...
1,en,en,1484396422_1483924666,https://www.stlucianewsonline.com/guyana-three...,https://www.thestar.com/news/world/europe/2020...,https://web.archive.org/web/www.stlucianewsonl...,https://web.archive.org/web/www.thestar.com/ne...,4.0,4.0,1.0,4.000000,3.666667,1.666667,1.333333,1484396422,1483924666,Share This On:\n\nPin 11 Shares\n\n(NEWS ROOM ...,Guyana: Three injured after car crashes into u...
2,en,en,1484698254_1483758694,https://www.teaparty.org/trump-brings-in-2020-...,https://www.timesofisrael.com/trump-says-he-do...,https://web.archive.org/web/www.teaparty.org/t...,https://web.archive.org/web/www.timesofisrael....,1.0,2.0,1.0,2.333333,2.333333,1.000000,1.333333,1484698254,1483758694,(Breitbart) – President Donald Trump welcomed ...,Trump Brings In 2020 At Mar-a-Lago: ‘We’re Goi...


In [29]:
leanDf2 = leanDf.rename(columns = {"id":"id2","text":"text2"})
merged = pd.merge(firstMerge, leanDf2, how="left", on="id2")
merged.head(3)

,url1_lang,url2_lang,pair_id,link1,link2,ia_link1,ia_link2,Geography,Entities,Time,Narrative,Overall,Style,Tone,id1,id2,text1,title1,text2,title
0,en,en,1484084337_1484110209,https://www.washingtonpost.com/local/virginia-...,https://www.washingtonpost.com/world/the_ameri...,https://web.archive.org/web/www.washingtonpost...,https://web.archive.org/web/www.washingtonpost...,4.0,4.0,1.0,4.000000,4.000000,1.666667,2.000000,1484084337,1484110209,"MARTINSBURG, W.Va. — A suspected drunken drive...",Virginia man arrested in fatal DUI crash in We...,"PORT-AU-PRINCE, Haiti — Haitian President Jove...",Haiti’s leader marks independence day amid sec...
1,en,en,1484396422_1483924666,https://www.stlucianewsonline.com/guyana-three...,https://www.thestar.com/news/world/europe/2020...,https://web.archive.org/web/www.stlucianewsonl...,https://web.archive.org/web/www.thestar.com/ne...,4.0,4.0,1.0,4.000000,3.666667,1.666667,1.333333,1484396422,1483924666,Share This On:\n\nPin 11 Shares\n\n(NEWS ROOM ...,Guyana: Three injured after car crashes into u...,BERLIN - A fire at a zoo in western Germany in...,Fire kills more than 30 animals at zoo in west...
2,en,en,1484698254_1483758694,https://www.teaparty.org/trump-brings-in-2020-...,https://www.timesofisrael.com/trump-says-he-do...,https://web.archive.org/web/www.teaparty.org/t...,https://web.archive.org/web/www.timesofisrael....,1.0,2.0,1.0,2.333333,2.333333,1.000000,1.333333,1484698254,1483758694,(Breitbart) – President Donald Trump welcomed ...,Trump Brings In 2020 At Mar-a-Lago: ‘We’re Goi...,"PALM BEACH, United States — US President Donal...","Trump says he does not expect war with Iran, ‘..."


In [30]:
mergedFull = merged.dropna(subset=["text1", "text2"])

In [31]:
mergedFull.shape

(4920, 20)

In [32]:
#send data from all languages 
mergedFull.to_csv("/shared/3/projects/benlitterer/localNews/NetworkMVP/allTrainData.csv", sep="\t")

In [33]:
mergedFullEn = mergedFull[(mergedFull["url1_lang"] == "en") & (mergedFull["url2_lang"] == "en")]

In [34]:
mergedFullEn.to_csv("/shared/3/projects/benlitterer/localNews/NetworkMVP/enTrainData.csv", sep="\t")